In [ ]:
!pip install -r requirements.txt

In [ ]:
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import os, sys

##### Importe dos dados

In [ ]:
# Importando dataset com ranking e índice de felicidade:
happines_dataset_path = "../data/happiness.csv"
happiness_df = pd.read_csv(happines_dataset_path)
happiness_df.info()

In [ ]:
# Importando dataset com taxa de homicidios:
homicide_rate_dataset_path = "../data/countries-by-intentional-homicide-rate.csv"
homicide_df = pd.read_csv(homicide_rate_dataset_path)
homicide_df.info()

##### 1 - Manipulação e Tratamento de dados

In [ ]:
# Renomeando as colunas:
happiness_df = happiness_df.rename(columns={"Countries": "country", "Happiness index, 2022": "happiness", "Global rank": "ranking", "Available data": "date_of_data"})
homicide_df = homicide_df[["Location","Rate","Count", "Year"]].rename(columns={"Location": "country", "Rate": "rate", "Count":"count", "Year": "date_of_data"})

# Garantindo a ordem do ranking de felicidade dos países no dataset de felicidade:
happiness_df = happiness_df.sort_values(by="happiness", ascending=False)
print("Dataset de Índice e Ranking de Felicidade")
display(happiness_df)

# Ordenando países por odem decrescente da taxa de homícidos:
homicide_df = homicide_df.sort_values(by="rate", ascending=True)
print("Dataset de Taxa de Homícidios")
display(homicide_df)

In [ ]:
# Trocando colunas com object para String:
happiness_df["country"] = happiness_df["country"].astype(str)
homicide_df["country"] = homicide_df["country"].astype(str)

In [ ]:
# Verifica se existem dados nulos (NaN) no dataset de felicidade:
if happiness_df.isnull().values.any():
    print("Existem dados nulos no conjunto de dados")
else:
    print("Não existem dados nulos no conjunto de dados")

sum_of_nulls = happiness_df.isnull().sum()
print("\nTotal de dados nulos por atributo:")
print(sum_of_nulls)

In [ ]:
# Verifica se existem dados nulos (NaN) no dataset de homicídios:
if homicide_df.isnull().values.any():
    print("Existem dados nulos no conjunto de dados")
else:
    print("Não existem dados nulos no conjunto de dados")

sum_of_nulls = homicide_df.isnull().sum()
print("\nTotal de dados nulos por atributo:")
print(sum_of_nulls)

In [ ]:
# Verificando quais períodos de data disponível no dataset de felicidade:
print("Períodos dos dados disponíveis no dataset de felicidade: ", happiness_df["date_of_data"].unique())

# Verificando quais períodos de data disponível no dataset de homicídio:
print("\nPeríodos dos dados disponíveis no dataset de homicídio: ", homicide_df["date_of_data"].unique())

##### 2 - Estística Descritiva

In [ ]:
# Total de países na pesquisa:
print("Existem %d países no dataset de felicidade. " % len(happiness_df["country"]))
print("Existem %d países no dataset de homicídios. " % len(homicide_df["country"]))

In [ ]:
# Média, moda e mediana do Índice de felicidade de 2022 (Happiness Index, 2022:
mean = happiness_df["happiness"].mean()
median = happiness_df["happiness"].median()
mode = max(set(list(happiness_df["happiness"].values)), key=list(happiness_df["happiness"].values).count)

print("A média do índice de felicidade é: %.2f" % mean)
print("A mediana do índice de felicidade é: %.2f" % median)
print("A moda do índice de felicidade é: %.2f" % mode)

In [ ]:
# Distribuição do Índice de Felicidade:
plt.title("Distribuição do Índice de Felicidade")
happiness_df["happiness"].hist()

In [ ]:
# Média, moda e mediana da taxa de homícidios:
mean = homicide_df["rate"].mean()
median = homicide_df["rate"].median()
mode = max(set(list(homicide_df["rate"].values)), key=list(homicide_df["rate"].values).count)

print("A média da taxa de homicídios é: %.2f" % mean)
print("A mediana da taxa de homicídios é: %.2f" % median)
print("A moda da taxa de homicídios é: %.2f" % mode)

In [ ]:
# Distribuição do Índice de Felicidade:
plt.title("Distribuição da Taxa de Homicídio")
homicide_df["rate"].hist()

- Pode-se perceber que existe uma maior quantidade de países com índice de felicidade alta (acima de 5), do que baixo. A disrirbuição não é Normal, pois tem um viés à esquerda.

- Percebe-se que no conjunto de dados de homicídios, a maior parte dos países possuem uma taxa de homicídio entre 0 a 20. Poucos países têm taxas muito altas, acima de 30.

##### 3 - Utilizando Groupby & Join

In [ ]:
group = happiness_df.groupby(["date_of_data"]).count()
display(group)

- Dos 134 países, 126 tiveram os dados coletados de 2013 a 2022, ou seja, cerca de 94% da amostra possui dados coletados durante o maior período de tempo.

In [ ]:
# Verificando quais países está presente nos dois datasets:
country_happ = list(happiness_df.country)
country_homi = list(homicide_df.country)

equal_countries = []

for c_01 in country_homi:
    for c_02 in country_happ:
        if c_01 == c_02:
            equal_countries.append(c_01)
print(len(equal_countries))

- Apenas 113 países, dos 134 no dataset de felicidade, estão também presentes no dataset de homícidios. Portanto, o join será feito apenas com estes 113 países.

In [ ]:
# Utilizando Join para realizar o cruzamento entre os datasets de felicidade e homicídios, sendo o menor (de felicidade) a base:
joined_df = happiness_df.set_index("country").join(homicide_df.set_index("country"), how="left", lsuffix='_happiness', rsuffix='_homicide')
display(joined_df)

- Como verificado anteriormente, apenas 113 paíes do dataset de felicidade também estão no dataset de homicídios. Desta forma, as linhas dos países que não estão em homicídios, ficaram com os valores NaN (Not a Number).

In [ ]:
# Removendo países que ficaram com dados faltantes:
joined_df = joined_df.dropna()
display(joined_df)